<img src="logo_UCLA_blue_boxed.svg" alt="ucla logo" width="150" align="left"/>

### **MAE 150A - Intermediate Fluid Mechanics**

<hr />

# Inspection of a steady flow field

In this notebook we will inspect a pre-computed steady flow field. First, we will look at some of the field quantities and get some intuition for them. Then, we will look at some flow lines.

### Set up the module

In [ ]:
using Pkg
pkg"update"

In [ ]:
using MAE150A

### Load the flow from file

We will use the flow from a pre-computed CFD solution. This is the steady state flow past a NACA 4415 airfoil at 5 degrees angle of attack at Reynolds number 1000.

In [ ]:
filename = "NACA4415_5deg.jld"
state, f, sys, body = load_ns_solution(filename)

Get the flow fields associated with the flow state: velocity (u), vorticity (ω), and streamfunction (ψ)

In [ ]:
u, ω, ψ = get_flowfield(state,sys)

### Plotting the flow field

In [ ]:
xg, yg = coordinates(ω,sys.grid)

Plot the vorticity field

In [ ]:
plot(xg,yg,ω,levels=range(-15,15,length=30), color = :RdBu,clim=(-15,15),ylim=(-1,1),xlim=(-1,4),size=(900,300))
plot!(body)

Plot the streamlines

In [ ]:
plot(xg,yg,ψ,levels=range(-1,2,length=31), color = :black, ylim = (-1,1),xlim = (-1,4),size=(900,600))
plot!(body)

Plot the velocity field components

In [ ]:
xue, yue, xve, yve = coordinates(u,sys.grid)

In [ ]:
ps1 = plot(xue,yue,u.u,levels=range(-0.1,1.5,length=31), color = :RdBu,clim=(-0.1,1.5),
    ylim=(-1,1),xlim=(-1,4),title="u component")
plot!(ps1,body)
ps2 = plot(xve,yve,u.v,levels=range(-0.15,0.15,length=31), color = :RdBu,clim=(-0.15,0.15),
    ylim=(-1,1),xlim=(-1,4),title="v component")
plot!(ps2,body)
plot(ps1,ps2,layout=(2,1),size=(600,600))

### Flow lines

#### Trajectory of a particle

In [ ]:
# Initial location of the particle
pts = [[-1,-0.5],[-1,0],[-1,0.25]]

# Time range of the trajectory
Tmax = 5.0

# Time step size
Δt = 0.001

# Compute the trajectory...
sol_array = compute_trajectory(u,sys,pts,Tmax,Δt);

Plot the trajectories

In [ ]:
ps = plot(body,xlim=(-1,4))
for sol in sol_array
    plot!(ps,sol[1,:],sol[2,:],linewidth=2)
end
plot(ps)

Evaluate the velocity components along a trajectory

In [ ]:
particle_number = 1

utraj = []
vtraj = []

# This loops through the particle's positions along the trajectory and evaluates the velocity components along it
for x in sol_array[particle_number].u
    push!(utraj,ufield(u...))
    push!(vtraj,vfield(u...))
end

Plot these

In [ ]:
plot(sol_array[1][1,:],utraj,ylim=(0,1.5),xlabel=L"x",label=L"u_p",title="Particle velocity components along trajectory")
plot!(sol_array[1][1,:],vtraj,ylim=(-1.5,1.5),label=L"v_p",ylabel="Velocity")